Your FIFA project goes here:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./file_for_project/fifa21_training.csv")
data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Nationality', 'Club', 'BP',
       'Position', 'Team & Contract', 'Height',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object', length=102)

In [3]:
data.columns = [e.lower().replace(' ' , '_') for e in data.columns] #unificamos formato nombres
data = data.dropna()

columns = data.columns.tolist()

data = data.drop(columns=["unnamed:_0","id",'name','nationality','team_&_contract'
        ,'club','joined','loan_date_end','contract'])

# A position la convertiremos en número de posiciones en las que puede jugar
# Qué es growth?
# Club, convertir a si es de gran liga o otro
# Wage, release_clause --> Quitar el símbolo de moneda, y cambiar la K y la M por los correspondientes 0.
# volleys, curve, agility, jumping, balance, interceptions, positioning, vision, composure, sliding_tackle  --> Son con decimales. Lo podemos dejar?
# w/f, sm --> Quitar el espacio y la estrella. Quedarnos sólo el indice [0]
# a/w, d/w, ir --> convertir High/Medium/Low en 1,2,3
# hits --> Quitar las comillas de string y K y M --> Se ha metido en la función dinero
# a partir de ls y hasta gk, mirar qué hacer con los símbolos +

In [4]:
#Funciones

def dinero(x):
    if '.' in x and 'K' in x:
        x=x.replace('K','00')
        x=x.replace('.','')
    else:
        x=x.replace('K','000')
    
    if '.' in x and 'M' in x:
        x=x.replace('M','00000')
        x=x.replace('.','')
    else:
        x=x.replace('M','000000')
        
    x=x.replace('€','')
    x=x.replace('$','')
    x=int(x)
    return x

def estrella(x):
    return int(x[0])

def himelo(x):
    if x == 'High':
        return 3
    if x == 'Medium':
        return 2
    if x == 'Low':
        return 1
    
def peso(x):
    return int(x.split('l')[0])

def altura(x):
    metros = int(x[0])*0.3048*100
    centimetros = int(x[-2])*2.54
    return metros+centimetros

In [5]:
# Conversión de columnas relativamente directa

data['position'] = [a.count(' ')+1 for a in data['position']]
data['wage'] = [dinero(wage) for wage in data['wage']]
data['release_clause'] = [dinero(rc) for rc in data['release_clause']]
data['value'] = [dinero(value) for value in data['value']]
data['w/f'] = [estrella(x) for x in data['w/f']]
data['sm'] = [estrella(x) for x in data['sm']]
data['a/w'] = [himelo(x) for x in data['a/w']]
data['d/w'] = [himelo(x) for x in data['d/w']]
data['ir'] = [himelo(x) for x in data['ir']]
data['hits'] = [dinero(x) for x in data['hits']]
data['weight'] = [peso(w) for w in data['weight']]
data['height'] = [altura(h) for h in data['height']]


In [6]:
# Conversión en masa de algunas columnas

col_con_mas = ['ls','st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

for i in col_con_mas:
    data[i+'+'] = [int(x.split('+')[1]) for x in data[i]]
    data[i] = [int(x.split('+')[0]) for x in data[i]]

In [7]:
# Estudio de la información

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Matriz de correlaciones

#corr = data.corr()
#mask = np.triu(np.ones_like(corr, dtype=np.bool))
#f, ax = plt.subplots(figsize=(11, 9))
#cmap = sns.diverging_palette(220, 10, as_cmap=True)

#sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={'shrink':.5});

# Salvo para las variables de portero, no se aprecian correlaciones significativas

In [8]:
# Estandarizar columnas numéricas (Todas entre -1 y 1) --> Quizá es mejor quitando antes outliers

from sklearn.preprocessing import StandardScaler

num_cols = [col for col in data.columns if (data[col].dtype!='object')]

for col in num_cols[:-1]:
    data[col] = StandardScaler().fit_transform(data[col].values.reshape(-1,1))